In [1]:
"""
5 jul 2017
summary using incremental clustering on word embedding similarity
"""

'\n5 jul 2017\nsummary using incremental clustering on word embedding similarity\n'

In [136]:
import spacy
import settings
from operator import itemgetter
from collections import defaultdict
import numpy as np
from scipy.linalg import norm
from tqdm import tqdm
import logging
import time

logging.basicConfig(format='%(asctime)s - %(name)s : %(message)s', level=logging.INFO, datefmt='%d-%m-%Y %H:%M:%S %Z')
logging.Formatter.converter = time.gmtime

In [143]:
nlp = spacy.load('en', parser=False, tagger=False, entity=False)

In [137]:
# load pre-trained vectors
from gensim.models import KeyedVectors

path = '/Users/mquezada/Downloads/glove.twitter.27B'
w2v = KeyedVectors.load_word2vec_format(path + '/glove.twitter.27B.200d.w2vformat.txt')


07-07-2017 14:53:32 UTC - gensim.models.keyedvectors : loading projection weights from /Users/mquezada/Downloads/glove.twitter.27B/glove.twitter.27B.200d.w2vformat.txt


KeyboardInterrupt: 

In [131]:
event_ids = settings.Datasets.mumbai_rape

with settings.engine.connect() as conn:
    query = f"SELECT d.* FROM document d join tweet t on d.tweet_id = t.tweet_id where t.event_id_id IN ({','.join(map(str, event_ids))}) ORDER BY t.created_at ASC;"
    cursor = conn.execute(query)
event = [row for row in cursor]
documents = list(map(itemgetter(1), event))
event[:1]

[(12816, 'Mumbai: In most rape cases, suspects knew victims or were related, says CID report, http://t.co/bFZCAhYXB1', 2, 8, 0, 1, 364612542829559809, None)]

In [139]:
threshold = 0.9
clusters = defaultdict(list)
centroids = dict()
vectors = defaultdict(list)
n_clusters = 0

doc_stream = nlp.pipe(documents)
first = next(doc_stream)
centroids[n_clusters] = first.vector / norm(first.vector)

for i, doc in tqdm(enumerate(doc_stream), total=len(documents)):
    docv = doc.vector / norm(doc.vector)
    max_ = 0
    idx_ = -1
    for j, centroid in centroids.items():
        sim = docv.dot(centroid)
        if sim > threshold and sim > max_:
            idx_ = j
            max_ = sim
    if idx_ == -1:
        # create new cluster
        clusters[n_clusters].append(i)
        centroids[n_clusters] = docv
        vectors[n_clusters].append(docv)
        n_clusters += 1
    else:
        # update existing cluster
        new_centroid = np.mean(vectors[idx_] + [docv], axis=0)
        centroids[idx_] = new_centroid / norm(new_centroid)
        clusters[idx_].append(i)
        

100%|█████████▉| 7955/7956 [00:08<00:00, 925.58it/s] 


In [140]:
len(clusters)

303

In [150]:
for token in doc:
    print(token.lower_)

rt
@irexmedia
:
two
more
arrests
made
in
gang
rape
of
photojournalist
in
india
#
journosafe
http://t.co/gwdxbwjqwe


In [122]:
for idx, cluster in clusters.items():
    print("cluster:", idx, "-", "size:", len(cluster))
    #sample_ = np.random.choice(cluster, replace=False, size=min(10, len(cluster)))
    #for i in sample_:
    #    print(documents[i])
    #print()
    
    cluster_docs = [event[i] for i in cluster]
    docs = sorted(cluster_docs, key=itemgetter(2), reverse=True)[:5]
    for d in docs:
        print(d[2], d[1])
    print()
    print()

cluster: 0 - size: 133
26 EVEN THE BEST LUXURY 5 STAR HOTEL IS NOTHING LIKE YOUR BED AT HOME
20 Las Vegas Luxury Vegas Airport Pickup Service Direct To Your Hotel- Reserve Today! Call 877-732-7603 http://t.co/23bOVnlE5f
12 A crane turned into a luxury hotel! Europe's most unique hotel experience is in #Amsterdam.  http://t.co/GsXLzhCVpl #travel #ttot
12 WIN: VIP BAFTA tickets, a luxury London hotel stay and an exclusive goody bag worth 500 http://t.co/n4wPVP0nBG
10 Go on a healing journey of relaxation and #rejuvenation @ITCHotels #Rajputana in India. http://t.co/iiuHAqxRyZ http://t.co/Tu0eEjdgV0


cluster: 1 - size: 1
2 #HD110 #epicchinaride Our hotel for the night. Living in luxury at the source of the Yangtze. http://t.co/yLXrnMT7


cluster: 2 - size: 1
2 BOOK at the #Palms A memorable luxury #LasVegas experience! http://t.co/3kNnY6cs http://t.co/jR6Q0a4d


cluster: 3 - size: 1
2 Karachi Lit Fest: Session on Social Satire held at Beach Luxury Hotel, Feb 2013 ...  http://t.co/AasazoT

In [134]:
# create html

footer = '<script async src="http://platform.twitter.com/widgets.js" charset="utf-8"></script>'

with open('event_embed.html', 'w') as f:
    for idx, cluster in sorted(clusters.items(), key=lambda x: len(x[1]), reverse=True)[:25]:
        f.write('<div style="width:100%"><div style="margin: 0 auto; width: 500px;">')
        f.write(f'<h1>Cluster {idx} - Size {len(cluster)}</h1>')

        cluster_docs = [event[i] for i in cluster]
        docs = sorted(cluster_docs, key=itemgetter(2), reverse=True)[:3]
        for d in docs:
            tweet_id = d[-2]
            f.write(f'<blockquote class="twitter-tweet" data-lang="en"><a href="https://twitter.com/asdf/status/{tweet_id}"></a></blockquote>')
        f.write("</div></div>")
        f.write(footer)

In [151]:
import scipy

In [152]:
%timeit scipy.random.sample()

The slowest run took 4014.87 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 425 ns per loop
